In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {"User-Agent": "Mozilla/5.0"}
base_url = "https://the-leader.jp/interview/page/{}/"
data = []

# 1〜17ページをループ
for page in range(1, 18):
    url = base_url.format(page)
    print(f"取得中: {url}")
    
    res = requests.get(url, headers=headers)
    if res.status_code != 200:
        print(f"❌ 取得失敗: {url}")
        continue

    soup = BeautifulSoup(res.text, "html.parser")
    articles = soup.select("div.tl_article")

    for article in articles:
        a_tag = article.select_one("a[href]")
        h2_tag = article.select_one("h2.tl_title")

        if a_tag and h2_tag:
            article_url = a_tag["href"]
            title = h2_tag.get_text(strip=True).replace("\n", "")
            data.append({
                "URL": article_url,
                "タイトル": title
            })
    time.sleep(1)  
# CSV保存
df = pd.DataFrame(data)
df.to_csv("the_leader_articles_all.csv", index=False, encoding="utf-8-sig")
print("✅ 全ページのCSV保存完了")

取得中: https://the-leader.jp/interview/page/1/
取得中: https://the-leader.jp/interview/page/2/
取得中: https://the-leader.jp/interview/page/3/
取得中: https://the-leader.jp/interview/page/4/
取得中: https://the-leader.jp/interview/page/5/
取得中: https://the-leader.jp/interview/page/6/
取得中: https://the-leader.jp/interview/page/7/
取得中: https://the-leader.jp/interview/page/8/
取得中: https://the-leader.jp/interview/page/9/
取得中: https://the-leader.jp/interview/page/10/
取得中: https://the-leader.jp/interview/page/11/
取得中: https://the-leader.jp/interview/page/12/
取得中: https://the-leader.jp/interview/page/13/
取得中: https://the-leader.jp/interview/page/14/
取得中: https://the-leader.jp/interview/page/15/
取得中: https://the-leader.jp/interview/page/16/
取得中: https://the-leader.jp/interview/page/17/
✅ 全ページのCSV保存完了


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# CSVファイルの読み込み（URLのリスト）
df = pd.read_csv("the_leader_articles_all.csv")
urls = df["URL"].tolist()

result = []

for url in urls:
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")

        box = soup.find("div", class_="box29")
        if not box:
            continue

        # 初期値
        company_name = ceo_name = site_url = ""

        for p in box.find_all("p"):
            text = p.get_text(strip=True)
            if "企業名" in text:
                company_name = text.split("：")[-1].strip()
            elif "代表者" in text:
                ceo_name = text.split("：")[-1].strip()
            elif "URL" in text:
                a = p.find("a")
                if a:
                    site_url = a["href"].strip()
        time.sleep(1)

        result.append({
            "企業名": company_name,
            "代表者": ceo_name,
            "URL": site_url,
            "インタビュー記事URL": url
        })
        

    except Exception as e:
        print(f"エラー: {url} -> {e}")

# CSVに保存
df_out = pd.DataFrame(result)
df_out.to_csv("the_leader_company_info.csv", index=False, encoding="utf-8-sig")
print("✅ 完了しました。")

✅ 完了しました。
